In [1]:
import pandas as pd
import seaborn as sns
import glob
import os
import anndata
import scipy
import scipy.cluster.hierarchy as sch
import numpy as np
# import scanpy as sc
from pathlib import Path
import multiprocessing
import scipy.spatial.distance as ssd
import scipy.cluster.hierarchy as sch
from sklearn.metrics import adjusted_mutual_info_score
import matplotlib.pyplot as plt
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [8]:
donor = 'common'

In [2]:
good_cov = 10 # coverage cutoff for single CpG site
good_cov_portion = 0.8 # consider the CpG site only if these many samples have good coverage criteria
good_mc_frac = 0.8 # the methylation level of CpG sites shoud be  >good_mc_frac  or  <1-good_mc_frac
good_mc_frac_portion = 0.9999 # consider the CpG site only if these many samples satisfy good mc frac criteria

pdist_thresh = 20

methylpy_results = glob.glob(f'/gale/netapp/home/wtian/hba/analysis/DMR/DMR-results/MajorType/MajorType.*_rms_results.tsv.gz')
name = f'celltype.MajorType.donor.common'

In [3]:
def parallelize_dataframe(dfreader, func, n_cores=20):
    with multiprocessing.Pool(n_cores) as pool:
        newdf = pd.concat(pool.map(func, dfreader))
    return newdf

In [4]:
# class DFProcessor:
#     def __init__(self, samples, good_cov, good_cov_portion, good_mc_frac, good_mc_frac_portion):
#         self.samples = samples
#         self.good_cov = good_cov
#         self.good_cov_portion = good_cov_portion
#         self.good_mc_frac = good_mc_frac
#         self.good_mc_frac_portion = good_mc_frac_portion
        
#     def process_df(self, df):
#         mcdf = df.loc[:, df.columns.str.startswith('mc_h19')].clip(0)
#         mcdf.columns = mcdf.columns.str.replace('mc_','',1)
#         covdf = df.loc[:, df.columns.str.startswith('h_h19')].clip(0)
#         covdf.columns = covdf.columns.str.replace('h_','',1)
#         fracdf_0 = mcdf/covdf
        
        
#         mcdf = mcdf[self.samples]
#         covdf = covdf[self.samples]
#         fracdf = fracdf_0[self.samples]
        
#         sel1 = (covdf>self.good_cov).mean(1)>self.good_cov_portion # good coverage
#         sel21 = ((fracdf-0.5).abs()>self.good_mc_frac-0.5).mean(1)>self.good_mc_frac_portion #good mc frac
#         sel22 = fracdf.max(1)-fracdf.min(1) > (2-self.good_mc_frac)/2  #good mc frac
#         sel = sel1&sel21&sel22
        
#         fracdf_0 = fracdf_0[sel].copy()
#         fracdf_0.columns = 'frac_'+fracdf_0.columns

        
#         tmp = pd.concat([df.loc[sel, ['chr','pos']],fracdf_0,
#                        df.loc[sel, df.columns.str.startswith('mc_h19')|df.columns.str.startswith('h_h19')]], axis=1)
        
        
# #         for col in ['chr','pos']:
# #             fracdf_0[col] = df[sel][col]
# #         fracdf_0['cell types'] = ':'.join(self.samples)
#         tmp['cell types'] = ':'.join(self.samples)

#         return tmp
        

In [5]:
class DFProcessor:
    def __init__(self, samples, good_cov, good_cov_portion, good_mc_frac, good_mc_frac_portion):
        self.samples = pd.Index(samples)
        self.good_cov = good_cov
        self.good_cov_portion = good_cov_portion
        self.good_mc_frac = good_mc_frac
        self.good_mc_frac_portion = good_mc_frac_portion
        
    def process_df(self, df):
        mcdf = df.loc[:, (df.columns!='mc_class') & df.columns.str.startswith('mc_') ].clip(0)
        mcdf.columns = mcdf.columns.str.replace('mc_','',1)
        covdf = df.loc[:, df.columns.str.startswith('h_')].clip(0)
        covdf.columns = covdf.columns.str.replace('h_','',1)
        fracdf_0 = mcdf/covdf
        
        
        mcdf = mcdf[self.samples]
        covdf = covdf[self.samples]
        fracdf = fracdf_0[self.samples]
        
        sel1 = (covdf>self.good_cov).mean(1)>self.good_cov_portion # good coverage
        sel21 = ((fracdf-0.5).abs()>self.good_mc_frac-0.5).mean(1)>self.good_mc_frac_portion #good mc frac
        sel22 = fracdf.max(1)-fracdf.min(1) > (2-self.good_mc_frac)/2  #good mc frac
        sel = sel1&sel21&sel22
        
        fracdf_0 = fracdf_0[sel].copy()
        fracdf_0.columns = 'frac_'+fracdf_0.columns

        
        tmp = pd.concat([df.loc[sel, ['chr','pos']],fracdf_0,
#                        df.loc[sel, df.columns.isin('mc_'+self.samples)|df.columns.isin('h_'+self.samples)]], axis=1)
                       df.loc[sel, ( (df.columns!='mc_class') & \
                                     (df.columns.str.startswith('mc_') | \
                                      df.columns.str.startswith('h_'))
                                   )]], axis=1)
        
        
#         for col in ['chr','pos']:
#             fracdf_0[col] = df[sel][col]
#         fracdf_0['cell types'] = ':'.join(self.samples)
        tmp['cell types'] = ':'.join(self.samples)

        return tmp
        

In [8]:
for x in pd.read_csv(methylpy_results[0],sep='\t',chunksize=20):
    break

In [9]:
x.columns[x.columns.str.startswith('methylation_level_')].str.replace('methylation_level_','')

Index([], dtype='object')

In [10]:
len(x.columns[x.columns.str.startswith('methylation_level_')].str.replace('methylation_level_',''))

0

In [6]:
# allsamples = [f'{donor}.Amy', f'{donor}.ASC', f'{donor}.ASC_Bergemann', f'{donor}.CA1',
#        f'{donor}.CA3', f'{donor}.CB', f'{donor}.CHD7', f'{donor}.DG', f'{donor}.EC',
#        f'{donor}.Foxp2', f'{donor}.HiF1', f'{donor}.HiF2', f'{donor}.L23_IT',
#        f'{donor}.L45_IT', f'{donor}.L4_IT', f'{donor}.L56_NP', f'{donor}.L5_ET',
#        f'{donor}.L5_IT', f'{donor}.L6b', f'{donor}.L6_CT', f'{donor}.L6_IT',
#        f'{donor}.L6_IT_Car3', f'{donor}.Lamp5', f'{donor}.Lamp5_LHX6',
#        f'{donor}.MGC', f'{donor}.MSN_D1', f'{donor}.MSN_D2', f'{donor}.ODC',
#        f'{donor}.OPC', f'{donor}.PC', f'{donor}.PKJ', f'{donor}.PN',
#        f'{donor}.Pvalb', f'{donor}.Pvalb_ChC', f'{donor}.Sncg', f'{donor}.Sst',
#        f'{donor}.Sst_Chodl', f'{donor}.SubCtx', f'{donor}.THM_Exc',
#        f'{donor}.THM_Inh', f'{donor}.THM_MB_MB', f'{donor}.THM_MB_THM',
#        f'{donor}.Vip', f'{donor}.VLMC']
allsamples = ['CA3', 'L6_IT', 'CB', 'MSN_D1', 'Amy', 'Sst', 'L6_CT', 'OPC', 'HiF1',
       'PC', 'L5_IT', 'L56_NP', 'Pvalb', 'CHD7', 'Lamp5_LHX6', 'Sncg',
       'SubCtx', 'THM_MB', 'Vip', 'L6b', 'L5_ET', 'MSN_D2', 'THM_Exc', 'L4_IT',
       'Foxp2', 'CA1', 'VLMC', 'THM_Inh', 'DG', 'ODC', 'L23_IT', 'PN', 'ASC',
       'Lamp5', 'EC', 'PKJ', 'HiF2', 'MGC', 'L6_IT_Car3', 'Pvalb_ChC']

In [7]:
samples_pool = [allsamples]
log = pd.DataFrame(index=allsamples)
log[0] = 0

roundid = 1
cpg_cands = pd.DataFrame()
while True:
    print('='*40,'round',roundid,'='*40)
    for samples in samples_pool:
        print(roundid, samples)
        for fn in sorted(methylpy_results):
            dfreader = pd.read_csv(fn, sep='\t', chunksize=5000)
            dfpr = DFProcessor(samples, good_cov, good_cov_portion, good_mc_frac, good_mc_frac_portion)
            tmp = parallelize_dataframe(dfreader, dfpr.process_df)
            tmp = tmp.set_index(['chr','pos'])
            tmp['round'] = roundid
            cpg_cands = pd.concat([cpg_cands, tmp])
#             cpg_cands = cpg_cands[~cpg_cands.index.duplicated()]
#             break
        
    


    binned = cpg_cands[[f'frac_{x}' for x in allsamples]]
    binned = binned.loc[~binned.index.duplicated()]
    binned = (binned>=good_mc_frac)*1+(binned<=1-good_mc_frac)*(-1)
    pdists = ssd.pdist(binned.T)
    distdf = pd.DataFrame(ssd.squareform(pdists), index=allsamples, columns=allsamples)
#     sns.clustermap(distdf,figsize=(15,15), vmax=pdist_thresh, vmin=pdist_thresh-0.01)

    
    
    log[roundid] = sch.cut_tree(sch.linkage(pdists,method='average'), height=pdist_thresh).flatten()
    if adjusted_mutual_info_score(log[roundid],log[roundid-1])==1:
        print('>'*10, 'nothing changes. stop iterating')
        break
        
        
    samples_pool = [x.index.tolist() for _,x in log.groupby(roundid) if len(x)>1]
    if len(samples_pool)==0:
        print('>'*10, 'all samples good. stop iterating')
        break


    print('>'*10, 'to process')
    for samples in samples_pool:
        print('\t',samples)

    roundid+=1



======================================== round 1 ========================================
1 ['CA3', 'L6_IT', 'CB', 'MSN_D1', 'Amy', 'Sst', 'L6_CT', 'OPC', 'HiF1', 'PC', 'L5_IT', 'L56_NP', 'Pvalb', 'CHD7', 'Lamp5_LHX6', 'Sncg', 'SubCtx', 'THM_MB', 'Vip', 'L6b', 'L5_ET', 'MSN_D2', 'THM_Exc', 'L4_IT', 'Foxp2', 'CA1', 'VLMC', 'THM_Inh', 'DG', 'ODC', 'L23_IT', 'PN', 'ASC', 'Lamp5', 'EC', 'PKJ', 'HiF2', 'MGC', 'L6_IT_Car3', 'Pvalb_ChC']
>>>>>>>>>> to process
	 ['L6_IT', 'Amy', 'L5_IT', 'L4_IT', 'L23_IT']
	 ['MSN_D1', 'MSN_D2']
	 ['Sst', 'Pvalb']
	 ['Sncg', 'SubCtx', 'Vip']
======================================== round 2 ========================================
2 ['L6_IT', 'Amy', 'L5_IT', 'L4_IT', 'L23_IT']
2 ['MSN_D1', 'MSN_D2']
2 ['Sst', 'Pvalb']
2 ['Sncg', 'SubCtx', 'Vip']
>>>>>>>>>> all samples good. stop iterating


In [9]:
cpg_cands.to_csv(f'./dataset_hierachy/celltype.MajorType.donor.common.candate-cpg.csv.gz')